In [ ]:
!pip install -q openai langchain langchain-openai

In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#기본 LLM 로드를 위한 라이브러리 호출
from langchain.llms import OpenAI

#채팅 LLM 로드를 위한 라이브러리 호출
from langchain.chat_models import ChatOpenAI

In [ ]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'

-------

#### Davinch-003 모델 설정하기

In [ ]:
davinch3 = OpenAI(
    model_name="text-davinci-003",
    max_tokens = 1000
)

#### (1) 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer")

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

In [ ]:
#to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능
print(string_prompt_value.to_string())

tell me a joke about soccer


In [ ]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer')])

In [ ]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

#### (2) 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- GPT-3와 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)


In [ ]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [ ]:
print(davinch3(
    prompt_template.format(
        재료 = '양파, 계란, 사과, 빵'
    )
))


추천하는 요리는 아래와 같아.

1. 양파 계란 볶음: 양파, 계란, 식용유, 소금 간장을 사용해서 만든 볶음

레시피

재료
- 양파: 1개 (썰어서)
- 계란: 2개 (두개로 쪼개서) 
- 식용유: 1큰술 
- 소금: 약간 
- 간장: 1큰술

만드는 방법
1. 양파를 썰어서 그릇에 담아 놓아. 
2. 계란을 두개로 쪼개서 그릇에 넣어줍니다.
3. 식용유, 소금, 간장을 넣어줍니다.
4. 모두 골고루 저어줍니다.
5. 볶음팬에 양념을 넣고 중간 불로 내열합니다.
6. 양파를 넣고 볶아줍니다.
7. 계란을 넣고 볶아줍니다. 
8. 다 볶은 후 더 볶고 마무리합니다. 

2. 사과 빵 디저트: 사과와 빵을 사용해서 만든 디저트 

레시피

재료
- 사과: 1개 (썰어서)
- 빵: 1~2장 (썰어서) 
- 버터: 2큰술 
- 설탕: 약간 

만드는 방법
1. 사과를 썰어서 그릇에 담아 놓아. 
2. 빵을 썰어서 그릇에 넣어줍니다.
3. 버터를 넣고 설탕을 약간 넣어줍니다.
4. 모두 골고루 저어줍니다.
5. 그릇에 담고 디저트 용기에 담아줍니다. 
6.


-  ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# ChatGPT 모델을 로드합니다.
chatgpt = ChatOpenAI(temperature=0)

#ChatGPT에게 역할을 부여합니다.(위에서 정의한 Template 사용)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

#사용자가 입력할 매개변수 template을 선언합니다.
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿을 삽입합니다.
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#ChatGPT API에 ChatPromptTemplate을 입력할 때, human message의 매개변수인 '재료'를 할당하여 전달합니다.
#이와 같은 방식을 통해 ChatGPT는 ChatPromptTemplate의 구성요소인 system message와 human message를 전달받아, 대답 생성에 활용합니다.
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

가지고 있는 재료로 만들 수 있는 요리 중 하나는 "양파 계란말이"입니다. 이 요리는 양파와 계란을 함께 볶아서 말아먹는 간단하면서도 맛있는 요리입니다. 아래는 양파 계란말이의 레시피입니다.

[양파 계란말이 레시피]
1. 양파를 깍두기 형태로 썰어줍니다.
2. 팬에 식용유를 두르고 양파를 넣어 중간 불에서 볶아줍니다.
3. 양파가 투명해질 때까지 볶은 후 계란을 풀어 넣어줍니다.
4. 계란이 익을 때까지 저어가며 볶아줍니다.
5. 양파와 계란이 잘 섞이고 익었으면 불을 끄고 소금과 후추로 간을 해줍니다.
6. 빵과 함께 내놓고 맛있게 즐기면 됩니다.

이렇게 양파 계란말이를 만들어보세요. 맛있게 드실 수 있을 거예요!


###(3) Few-shot 예제를 통한 프롬프트 템플릿

Few-shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론입니다.

LLM 역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력합니다.

내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있습니다.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [ ]:
print(davinch3.predict("호날두로 삼행시 만들어줘"))



호날두 떠나면 혼자 늘 외로워
내 앞에는 다른 누구도 없는데
나는 가끔 슬플 때도 있어
그때는 네가 언제나 곁에 있던 걸

호날두 내 곁에 있던 기억이 나
어제 이렇게 같이 웃었던 걸
호날두 내게 넌 다시 오지 않을까
널 기다려 할 땐 마음 너무 힘들어

난 널 보고 싶어 널 보고 싶어
사랑하는 너를 다시 만나고 싶어
널 보면 마음이 흔들려 흔들려
잊지 못할 추억이 떠오르네

호날두 내 곁에 있던 기억이 나
어제 이렇게 같이 웃었던 걸
호날두 내게 넌 다시 오지 않을까
널 기다려 할 땐 마음 너무 힘들어

비가 오면 내 마음이 슬프게 변하네
나의 가슴 속에 널 찾아가네
난 널 보고 싶어 널 보고 싶어
사랑하는 너를 다시 만나고 싶어

난 널 보고 싶어 널 보고 싶어
사랑하는 너를 다시 만나고 싶어
널 보면 마음이 흔들려 흔들려
잊지 못할 추억이 떠오르네

내게 다시 너를 보내줘


In [ ]:
print(davinch3(
    prompt.format(input="호날두로 삼행시 만들어줘")
))



호: 호주에서 왔어
날: 날개가 펄럭이고
두: 두근두근 마음이 뛰어대는 날


### (4) Example Selector를 이용한 동적 Few-shot 러닝

Few-shot 예제를 동적으로 입력하고 싶은 경우, Example Selector를 활용할 수 있습니다.

LLM이 여러 작업을 수행하도록 만들되 내가 원하는 범위의 대답을 출력하도록 하려면 사용자의 입력에 동적으로 반응해야 합니다.이와 동시에, 예제를 모두 학습시키는 것이 아니라 적절한 예시만 포함하도록 함으로써 입력 prompt의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있습니다.

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [ ]:
!pip install chromadb
!pip install tiktoken

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

In [ ]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [ ]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [ ]:
query = "큰 비행기"

print(davinch3(
    similar_prompt.format(단어=query)
))

 작은 비행기


### (5) Output Parser를 활용한 출력값 조정

LLM의 답변을 내가 원하는 형태로 고정하고 싶다면 OutputParser 함수를 활용할 수 있습니다. 리스트, JSON 형태 등 다양한 형식의 답변을 고정하여 출력할 수 있습니다.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
output_parser = CommaSeparatedListOutputParser()

In [ ]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [ ]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
model = OpenAI(temperature=0)

In [ ]:
_input = prompt.format(주제="영화")
output = model(_input)

In [ ]:
output

'\n\n아이 스타일, 스타 워즈: 깨어난 포스, 엔드게임, 스파이더맨: 홈커밍, 엑스맨: 데이즈 오브 퓨처, 어벤져스: 엔드게임'

In [ ]:
output_parser.parse(output)

['아이 스타일',
 '스타 워즈: 깨어난 포스',
 '엔드게임',
 '스파이더맨: 홈커밍',
 '엑스맨: 데이즈 오브 퓨처',
 '어벤져스: 엔드게임']